[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [ ]:
import torch

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [ ]:
outputs = model(torch.randn(600, 784))
outputs.shape

torch.Size([600, 10])

In [ ]:
print(outputs[0][:])

tensor([-0.1676,  0.1312,  0.2731,  0.0290,  0.1970, -0.4047,  0.1382,  0.0532,
         0.2776,  0.2289], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [ ]:
model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [ ]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

((70000, 784), (70000,))

In [ ]:
# import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:60000] / 255.
X_test =x_2[60000:] / 255.
y_train = y_2[:60000].astype(np.int32)
y_test = y_2[60000:].astype(np.int32)



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
# X_train

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print(X)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 350
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/350 Loss 1.80185
Epoch 20/350 Loss 1.42818
Epoch 30/350 Loss 1.16257
Epoch 40/350 Loss 1.00993
Epoch 50/350 Loss 0.88668
Epoch 60/350 Loss 0.80146
Epoch 70/350 Loss 0.73098
Epoch 80/350 Loss 0.67588
Epoch 90/350 Loss 0.63143
Epoch 100/350 Loss 0.59478
Epoch 110/350 Loss 0.56501
Epoch 120/350 Loss 0.54098
Epoch 130/350 Loss 0.51772
Epoch 140/350 Loss 0.49700
Epoch 150/350 Loss 0.47850
Epoch 160/350 Loss 0.46187
Epoch 170/350 Loss 0.44682
Epoch 180/350 Loss 0.43311
Epoch 190/350 Loss 0.42056
Epoch 200/350 Loss 0.40902
Epoch 210/350 Loss 0.39835
Epoch 220/350 Loss 0.38844
Epoch 230/350 Loss 0.37921
Epoch 240/350 Loss 0.37058
Epoch 250/350 Loss 0.36249
Epoch 260/350 Loss 0.35486
Epoch 270/350 Loss 0.34767
Epoch 280/350 Loss 0.34087
Epoch 290/350 Loss 0.33443
Epoch 300/350 Loss 0.32831
Epoch 310/350 Loss 0.32249
Epoch 320/350 Loss 0.31695
Epoch 330/350 Loss 0.31166
Epoch 340/350 Loss 0.30661
Epoch 350/350 Loss 0.30177


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9588

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 1500
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/1500 Loss 1.91606
Epoch 20/1500 Loss 1.54748
Epoch 30/1500 Loss 1.24995
Epoch 40/1500 Loss 1.05559
Epoch 50/1500 Loss 0.93014
Epoch 60/1500 Loss 0.82928
Epoch 70/1500 Loss 0.76232
Epoch 80/1500 Loss 0.70510
Epoch 90/1500 Loss 0.65683
Epoch 100/1500 Loss 0.61695
Epoch 110/1500 Loss 0.58338
Epoch 120/1500 Loss 0.55472
Epoch 130/1500 Loss 0.53017
Epoch 140/1500 Loss 0.50891
Epoch 150/1500 Loss 0.48958
Epoch 160/1500 Loss 0.47203
Epoch 170/1500 Loss 0.45616
Epoch 180/1500 Loss 0.44174
Epoch 190/1500 Loss 0.42857
Epoch 200/1500 Loss 0.41646
Epoch 210/1500 Loss 0.40529
Epoch 220/1500 Loss 0.39495
Epoch 230/1500 Loss 0.38532
Epoch 240/1500 Loss 0.37634
Epoch 250/1500 Loss 0.36793
Epoch 260/1500 Loss 0.36003
Epoch 270/1500 Loss 0.35260
Epoch 280/1500 Loss 0.34558
Epoch 290/1500 Loss 0.33895
Epoch 300/1500 Loss 0.33265
Epoch 310/1500 Loss 0.32668
Epoch 320/1500 Loss 0.32100
Epoch 330/1500 Loss 0.31558
Epoch 340/1500 Loss 0.31041
Epoch 350/1500 Loss 0.30546
Epoch 360/1500 Loss 0.30073
E

0.977

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [ ]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(500, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.3565,  1.1012, -0.2018,  ...,  0.9919, -1.0762,  0.8937],
        [-0.0618, -0.3041, -0.7541,  ...,  0.8137, -1.1066, -0.7269],
        [ 0.8396,  1.9817,  0.8287,  ..., -0.1057,  0.3610,  1.3960],
        ...,
        [ 0.3159,  0.0116,  0.7946,  ..., -0.4346, -0.0328, -0.2375],
        [ 1.2079, -2.2098,  0.3712,  ..., -0.5540, -0.3757,  0.3369],
        [ 1.3936,  0.4179,  0.4427,  ...,  1.0251,  1.6883, -0.0341]])


torch.Size([500, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.80042
Epoch 20/100 Loss 1.44855
Epoch 30/100 Loss 1.18647
Epoch 40/100 Loss 1.00444
Epoch 50/100 Loss 0.89447
Epoch 60/100 Loss 0.79948
Epoch 70/100 Loss 0.73598
Epoch 80/100 Loss 0.68085
Epoch 90/100 Loss 0.63504
Epoch 100/100 Loss 0.59709


0.9281

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [ ]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [ ]:
model = ModelCustom2(784, 100, 10).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.61140
Epoch 20/100 Loss 1.17541
Epoch 30/100 Loss 0.97012
Epoch 40/100 Loss 0.86633
Epoch 50/100 Loss 0.78405
Epoch 60/100 Loss 0.72089
Epoch 70/100 Loss 0.67117
Epoch 80/100 Loss 0.63201
Epoch 90/100 Loss 0.60033
Epoch 100/100 Loss 0.57409


0.9117

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [ ]:
model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
model.fc1

Linear(in_features=784, out_features=100, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[-0.0242,  0.0021,  0.0176,  ..., -0.0336,  0.0020,  0.0276],
        [ 0.0087,  0.0264,  0.0148,  ..., -0.0270,  0.0226,  0.0109],
        [-0.0031, -0.0064, -0.0013,  ...,  0.0343,  0.0168,  0.0337],
        ...,
        [ 0.0045,  0.0308,  0.0144,  ..., -0.0100, -0.0259,  0.0305],
        [-0.0041, -0.0255,  0.0070,  ...,  0.0045, -0.0154,  0.0098],
        [-0.0106, -0.0082,  0.0334,  ...,  0.0010,  0.0220,  0.0106]],
       device='cuda:0', requires_grad=True)

In [ ]:
model.fc1.bias

Parameter containing:
tensor([ 0.0387,  0.0041,  0.0606,  0.0189,  0.0038,  0.0597,  0.0370,  0.0164,
         0.0435,  0.0296,  0.0101, -0.0244,  0.0711,  0.0083, -0.0255, -0.0142,
         0.0796,  0.0341,  0.0126,  0.0849,  0.0348, -0.0262,  0.0629,  0.0505,
         0.0157, -0.0331,  0.0089,  0.0358, -0.0023,  0.0761, -0.0121, -0.0230,
        -0.0494, -0.0106, -0.0401,  0.0433, -0.0140,  0.0412, -0.0090,  0.0406,
        -0.0070,  0.0441, -0.0031, -0.0359, -0.0505,  0.0183,  0.0318,  0.0594,
        -0.0394,  0.0092,  0.0046, -0.0073, -0.0190, -0.0106,  0.0186, -0.0167,
        -0.0101,  0.0710,  0.0360, -0.0011,  0.0529, -0.0381,  0.0713,  0.0413,
         0.0012,  0.0613,  0.0153,  0.0281,  0.0599, -0.0387, -0.0231, -0.0055,
        -0.0166, -0.0257,  0.0488,  0.0519, -0.0254, -0.0140, -0.0206,  0.0031,
        -0.0122, -0.0025,  0.0772,  0.0052,  0.0097,  0.0421, -0.0466, -0.0211,
        -0.0262,  0.0959,  0.0098,  0.0740,  0.0467, -0.0215,  0.0439, -0.0191,
        -0.0278,  

Es posible sobreescribir una capa de la siguiente manera

In [ ]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [ ]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
)

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.